In [1]:
%pip install scipy
%pip install numpy
%pip install pandas
%pip install networkx
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.model_selection import train_test_split
from typing import Tuple, Mapping, Any

In [3]:
df = pd.read_csv("res\\rating.csv")

In [4]:
def split_by_target_column_value(df: pd.DataFrame, test_size: float, target_column: str, target_value: Any,
                                 random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Разделяет DataFrame на обучающий и тестовый наборы на основе определенного значения целевого столбца.
    Выдает в качестве тестового набора test_size от набора, в котором значение в target_column == target_value
    В тренировочном все остальное
    Параметры:
        df (pd.DataFrame): DataFrame для разделения.
        test_size: Пропорция данных, которые следует включить в тестовый набор.
        target_column (str): Название целевого столбца для разделения.
        target_value (Any): Значение целевого столбца, используемое для разделения.
        random_state (int): Зерно для генератора случайных чисел.

    Возвращает:
        Tuple[pd.DataFrame, pd.Series, pd.DataFrame, pd.Series]: Данные для обучения; тестовые данные для целевой колонки, которые не попали в data,
        тренировочные данные для целевой колоники, которые попали в data
    """
    grouped = df.groupby(by=target_column)
    train_df, test_df = train_test_split(grouped.get_group(target_value), test_size=test_size,
                                         random_state=random_state)
    data = pd.concat([group for ind, group in grouped if ind != target_value], ignore_index=True)
    data = pd.concat([train_df, data], ignore_index=True)
    return data, test_df, train_df

In [5]:
data, test1, train1 = split_by_target_column_value(df, test_size=0.5, target_column='userId', target_value=1)

In [6]:
data.head()

,userId,movieId,rating,timestamp
0,1,2288,4.0,2004-09-10 03:14:37
1,1,2648,3.5,2004-09-10 03:13:37
2,1,1222,3.5,2005-04-02 23:30:37
3,1,3037,3.5,2005-04-02 23:46:03
4,1,2683,3.5,2004-09-10 03:07:30


In [7]:
test1.head()

,userId,movieId,rating,timestamp
156,1,7045,3.5,2005-04-02 23:50:34
145,1,6242,3.5,2004-09-10 03:13:24
101,1,3081,4.0,2004-09-10 03:15:43
127,1,4911,4.0,2005-04-02 23:52:41
141,1,5898,3.5,2005-04-02 23:53:22


In [8]:
train1.head()

,userId,movieId,rating,timestamp
79,1,2288,4.0,2004-09-10 03:14:37
84,1,2648,3.5,2004-09-10 03:13:37
39,1,1222,3.5,2005-04-02 23:30:37
100,1,3037,3.5,2005-04-02 23:46:03
86,1,2683,3.5,2004-09-10 03:07:30


In [13]:
def get_rank(data: pd.DataFrame, nstart: Mapping,
               source: str, target: str, edge_attr: str) -> pd.DataFrame:
    """
    Вычисляет рейтинг страницы(hits) на основе заданного DataFrame и возвращает DataFrame с рангами каждого фильма с учетом персонализации.
    Параметры:
        data (pd.DataFrame): DataFrame для вычисления PageRank.
        nstart (Mapping): Словарь с начальными значениями
        source (str): Название столбца, содержащего исходные вершины ребер.
        target (str): Название столбца, содержащего целевые вершины ребер.
        edge_attr (str): Название столбца, содержащего атрибуты ребер.

    Возвращает:
        pd.DataFrame: DataFrame с результатами вычисления PageRank с учетом персонализации.
    """
    G = nx.Graph()

    # Создаем список ребер для добавления
    edges_to_add = [((row[source], 'source'), (row[target], 'target'), {'weight': row[edge_attr]}) for _, row in data.iterrows()]

    # Добавляем все ребра скопом
    G.add_edges_from(edges_to_add)
    
    # Преобразование персонализации в формат, подходящий для PageRank
    new_nstart = {(k, 'source'): v for k, v in nstart.items()}
    
    # Вычисление PageRank
    hubs, authorities = nx.hits(G, nstart=new_nstart)
    
    # Формирование DataFrame с результатами PageRank
    target_rank_generator = ((node[0], rank) for node, rank in authorities.items() if node[1] == 'target')
    return pd.DataFrame(target_rank_generator, columns=[target, 'hits'])


In [19]:
# Создаем тестовый DataFrame
test_df = pd.DataFrame({
    'source': [1, 2, 2, 2],
    'target': [2, 2, 3, 1],
    'edge_attr': [5, 5, 5, 1]
})

# Создаем тестовую персонализацию
personalization = {1: 0.9}

prediction_test = get_rank(test_df, nstart=personalization, source='source', target='target',
                                            edge_attr='edge_attr')
prediction_test

ValueError: `v0` must have shape (5,).

In [ ]:
prediction_movie_for_user1 = get_rank(data=data, personalization={1: 2}, source='userId', target='movieId',
                                            edge_attr='rating')

In [ ]:
prediction_movie_for_user1.head()

In [ ]:
prediction_movie_for_user1_without_famous = prediction_movie_for_user1[~(prediction_movie_for_user1['movieId'].isin(train1['movieId']))]
print(len(prediction_movie_for_user1_without_famous))
prediction_movie_for_user1_without_famous.head()

In [ ]:
# Топ фильмов по мнению hits
pred = prediction_movie_for_user1_without_famous.sort_values(by='pagerank', ascending=False).head(len(test1))
print(len(pred))
pred.head()

In [ ]:
def accuracy_without_order(true_labels, predicted_labels):
    true_set = set(true_labels)
    pred_set = set(predicted_labels)
    
    # Проверка на равенство множеств
    accuracy = len(true_set.intersection(pred_set)) / len(true_set)
    
    return accuracy

In [ ]:
print(len(test1['movieId']))
test1['movieId'].head()

In [ ]:
print(len(pred['movieId']))
pred['movieId'].head()

In [ ]:
print(accuracy_without_order(test1['movieId'], pred['movieId']))

In [ ]:
movie_df = pd.read_csv('res/movie.csv')

In [ ]:
prediction_moviename = pd.merge(movie_df, pred, on='movieId', how='inner')
prediction_moviename.sort_values(by='pagerank', ascending=False).head()

In [ ]:
test_moviename = pd.merge(movie_df, test1, on='movieId', how='inner')
print(len(test_moviename))
test_moviename.sort_values(by='rating', ascending=False).head()

In [ ]:
# Фильмы которые были угаданны
test_moviename[test_moviename['movieId'].isin(prediction_moviename['movieId'])]